In [3]:
import matplotlib.pyplot as plt
from os.path import join
import dask
import numpy as np  
import pandas as pd
from scipy.io import loadmat
from collections import namedtuple
import mrestimator as mre

import loren_frank_data_processing
from loren_frank_data_processing.core import get_data_filename, logger, get_epochs, get_data_structure
from loren_frank_data_processing.tetrodes import get_trial_time
from loren_frank_data_processing.neurons import get_spikes_dataframe, make_neuron_dataframe, convert_neuron_epoch_to_dataframe, _add_to_dict, _get_neuron_id
import loren_frank_data_processing.core

In [4]:
Animal = namedtuple('Animal', {'directory', 'short_name'})
conley = Animal('C:\\Users\janbe\Downloads\Conley.tar\Conley', 'con')
corriander = Animal('C:\\Users\janbe\Downloads\Corriander.tar\Corriander', 'Cor')
animals = {'con': Animal('con','C:\\Users\\janbe\\Downloads\\Conley.tar\\Conley'), 'Cor': Animal('Cor','C:\\Users\janbe\Downloads\Corriander.tar\Corriander')} 

In [5]:
#the following functions from 'loren_frank_data_processing'got modified, with changes indicated

def convert_neuron_epoch_to_dataframe_modified(tetrodes_in_epoch, animal, day, epoch):
    '''
    Given an neuron data structure, return a cleaned up DataFrame
    '''
    DROP_COLUMNS = ['ripmodtag', 'thetamodtag', 'runripmodtag',
                    'postsleepripmodtag', 'presleepripmodtag',
                    'runthetamodtag', 'ripmodtag2', 'runripmodtag2',
                    'postsleepripmodtag2', 'presleepripmodtag2',
                    'ripmodtype', 'runripmodtype', 'postsleepripmodtype',
                    'presleepripmodtype', 'FStag', 'ripmodtag3',
                    'runripmodtag3', 'ripmodtype3', 'runripmodtype3',
                    'tag', 'typetag', 'runripmodtype2',
                    'tag2', 'ripmodtype2', 'descrip']

    NEURON_INDEX = ['animal', 'day', 'epoch',
                    'tetrode_number', 'neuron_number']

    neuron_dict_list = [_add_to_dict(
        _convert_to_dict(neuron), tetrode_ind, neuron_ind)
        for tetrode_ind, tetrode in enumerate(
        tetrodes_in_epoch[0][0])
        for neuron_ind, neuron in enumerate(tetrode[0])
        #if neuron.size > 0
    ]
    try:
        return (pd.DataFrame(neuron_dict_list)
                  .drop(DROP_COLUMNS, axis=1, errors='ignore')
                  .assign(animal=animal)
                
                  .assign(day=day)
                  .assign(epoch=epoch)
                  .assign(neuron_id=_get_neuron_id
                         )
                # set index to identify rows
                  .set_index(NEURON_INDEX)
                  .sort_index())
    except AttributeError:
        logger.debug(f'Neuron info {animal}, {day}, {epoch} not processed')

def _convert_to_dict_modified(struct_array):
    try:
        return {name: struct_array[name].item().item()
                for name in struct_array.dtype.names
                if struct_array[name].item().size == 1}
    except TypeError:
        return {}
    #added in the AttributeError
    except AttributeError:
        return {}
    
def get_spike_indicator_dataframe_modified(neuron_key, animals, time_function=get_trial_time):
    time = time_function(neuron_key[:3], animals)
    spikes_df = get_spikes_dataframe(neuron_key, animals)
    time_index = None
    try:
        time_index = np.digitize(spikes_df.index.total_seconds(),
                             time.total_seconds())
 
        time_index[time_index >= len(time)] = len(time) -1
    #the exception is for empty data
    except AttributeError: 
        print('No spikes here; data is emtpy')
    
    #the following accounts for an empty time_index
    if time_index is not None:
        return (spikes_df.groupby(time[time_index]).sum().reindex(index=time, fill_value=0))
    else:
        return None
        
def make_neuron_dataframe_modified(animals):
    '''Information about all recorded neurons such as brain area.
    The index of the dataframe corresponds to the unique key for that neuron
    and can be used to load spiking information.
    Parameters
    ----------
    animals : dict of named-tuples
        Dictionary containing information about the directory for each
        animal. The key is the animal_short_name.
    Returns
    -------
    neuron_information : pandas.DataFrame
    '''
    #neuron_file_names = [(get_neuron_info_path(animals[animal]), animal)
     #                    for animal in animals]
    neuron_file_names = ['C:\\Users\janbe\Downloads\Conley.tar\Conley\concellinfo.mat', 'C:\\Users\janbe\Downloads\Corriander.tar\Corriander\Corcellinfo.mat']
    neuron_data = [(loadmat(file_name), animal_name) for animal_name, file_name in zip(animals.keys(), neuron_file_names)]

    return pd.concat([
        convert_neuron_epoch_to_dataframe(
            epoch, animal, day_ind + 1, epoch_ind + 1)
        for cellfile, animal in neuron_data
        for day_ind, day in enumerate(cellfile['cellinfo'].T)
        for epoch_ind, epoch in enumerate(day[0].T)
    ]).sort_index()

#modified for access with correct name
def get_tetrode_info_path_modified(animal):
    '''Returns the Matlab tetrode info file name assuming it is in the
    Raw Data directory.
    Parameters
    ----------
    animal : namedtuple
        First element is the directory where the animal's data is located.
        The second element is the animal shortened name.
    Returns
    -------
    filename : str
        The path to the information about the tetrodes for a given animal.
    '''
    filename = '{animal}tetinfo'.format(animal=animal.short_name)
    return filename

In [6]:
#new functions 

#sums 2DArray of trials over time, over all trials, such that one cummulative time-series results
def sum_time_series_dict(time_series_dict):
    
    num_bins = max([len(x) for x in time_series_dict.values() if x is not None])
    res = np.zeros(num_bins)
    for arr in time_series_dict.values():
        if arr is not None:
            res[:len(arr)] += arr
    return res

#generates dictionary of time-series, relative to neuron_key_list (indeces) given as input
def generate_spike_indicator_dict(neuron_key_list, animals):
    spike_indicator_dict = {}
    for neuron_key_str in neuron_key_list:
        animal_short_name, day_number, epoch_number, tetrode_number, neuron_number = neuron_key_str.split("_")
        neuron_key = (animal_short_name, int(day_number), int(epoch_number), int(tetrode_number), int(neuron_number))
        try:
            spike_indicator_array = get_spike_indicator_dataframe_modified(neuron_key, animals).values
        except AttributeError:
            spike_indicator_array = None
            print(f"No spike indicator data for neuron: {neuron_key}")
        spike_indicator_dict[neuron_key] = spike_indicator_array
    return spike_indicator_dict


def split_neuron_dataframe_informationally(df, split_cols):
    '''Splits a DataFrame into multiple DataFrames based on specified
    column values.
    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame to be split.
    split_cols : list of str
        The names of the columns to use for splitting the DataFrame.
    Returns
    -------
    dfs : dict
        A dictionary containing the split DataFrames. The keys are tuples
        containing the unique combinations of the specified column values.
    '''
    dfs = {}
    for idx, group in df.groupby(split_cols):
        dfs[idx] = group.copy()
    return dfs

def merge_dicts(dict_list):
    merged_dict = {}
    for dictionary in dict_list:
        merged_dict.update(dictionary)
    return merged_dict
